In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import string


In [2]:
path_training_small_truth = "../data/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small-truth.jsonl"
path_training_small = "../data/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small.jsonl"

In [3]:
df_texts = pd.read_json(path_training_small, lines=True)

In [4]:
df_truth = pd.read_json(path_training_small_truth, lines=True)

In [5]:
df_join_training_data = pd.concat([df_truth, df_texts], axis=1).reindex(df_truth.index)

In [6]:
df_join_training_data.head()

,id,same,authors,id,fandoms,pair
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"[1446633, 1446633]",6cced668-6e51-5212-873c-717f2bc91ce6,"[Guardians of Ga'Hoole, Hetalia - Axis Powers]","[I shift a bit, warily letting my eyes dart fr..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"[1446633, 1446633]",3c6c188a-db28-59aa-8c09-3d0f799ff579,"[Guardians of Ga'Hoole, Warriors]","[I shift a bit, warily letting my eyes dart fr..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,"[1446633, 1446633]",b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,"[Guardians of Ga'Hoole, Xiaolin Showdown]",[A single tear escaped me as I left. I did hav...
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"[1446633, 1446633]",e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,"[Hetalia - Axis Powers, Warriors]","[""Ja."" Ludwig kept his gaze upon her, solidly...."
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"[1446633, 1446633]",4fe541af-912e-5a86-81a5-94c6d3891509,"[Hetalia - Axis Powers, Xiaolin Showdown]","[And he did. Slowly, hesitantly...but coming f..."


In [7]:
df_join_training_data = df_join_training_data.loc[:,~df_join_training_data.columns.duplicated()]

In [8]:
df_join_training_data.head()

,id,same,authors,fandoms,pair
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Hetalia - Axis Powers]","[I shift a bit, warily letting my eyes dart fr..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Warriors]","[I shift a bit, warily letting my eyes dart fr..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Xiaolin Showdown]",[A single tear escaped me as I left. I did hav...
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"[1446633, 1446633]","[Hetalia - Axis Powers, Warriors]","[""Ja."" Ludwig kept his gaze upon her, solidly...."
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"[1446633, 1446633]","[Hetalia - Axis Powers, Xiaolin Showdown]","[And he did. Slowly, hesitantly...but coming f..."


In [9]:
df_join_training_data.dtypes

id         object
same         bool
authors    object
fandoms    object
pair       object
dtype: object

In [10]:
text = df_join_training_data.loc[0].pair

In [11]:
type(text), len(text)

(list, 2)

In [12]:
df_join_training_data[['text1','text2']] = pd.DataFrame(df_join_training_data.pair.tolist(), index= df_join_training_data.index)
df_join_training_data[['author1','author2']] = pd.DataFrame(df_join_training_data.authors.tolist(), index= df_join_training_data.index)

In [13]:
df_join_training_data = df_join_training_data.drop(columns=["pair", "fandoms","authors"])

In [14]:
df_join_training_data

,id,same,text1,text2,author1,author2
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al...",1446633,1446633
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of...",1446633,1446633
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha...",1446633,1446633
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...,1446633,1446633
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ...",1446633,1446633
...,...,...,...,...,...,...
52596,3926a1bd-6d33-5513-a694-12d7590443d4,False,Enjoy! And time stands still beneath the air o...,"""That a challenge, dick?"" Austin smirks back a...",1940652,4353578
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,False,"""I forgive you,"" she blurted out looking strai...","This is a KNB fanfic. KNB doesn""t belong to me...",2581924,5085069
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,False,"""What? Why? What""s happening? Where are we goi...","At the moment, Chris"" position on the swing se...",4819080,6983686
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,False,"""Gah!"" I growled. ""I need a password to access...",As you can see a lot of things has happened si...,6070317,4369540


In [15]:
df_join_training_data.to_csv("../data/temp/data.csv", index=False)

In [16]:
## Load csv

##### Word Punctuation feature

In [17]:
df_join_training_data = pd.read_csv("../data/temp/data.csv")

In [18]:
df_join_training_data

,id,same,text1,text2,author1,author2
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al...",1446633,1446633
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of...",1446633,1446633
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha...",1446633,1446633
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...,1446633,1446633
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ...",1446633,1446633
...,...,...,...,...,...,...
52596,3926a1bd-6d33-5513-a694-12d7590443d4,False,Enjoy! And time stands still beneath the air o...,"""That a challenge, dick?"" Austin smirks back a...",1940652,4353578
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,False,"""I forgive you,"" she blurted out looking strai...","This is a KNB fanfic. KNB doesn""t belong to me...",2581924,5085069
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,False,"""What? Why? What""s happening? Where are we goi...","At the moment, Chris"" position on the swing se...",4819080,6983686
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,False,"""Gah!"" I growled. ""I need a password to access...",As you can see a lot of things has happened si...,6070317,4369540


In [19]:
texts = np.hstack([df_join_training_data['text1'].values, df_join_training_data['text2'].values])


In [20]:
vocab = "!.?$()"

In [21]:
vect = TfidfVectorizer(vocabulary=string.punctuation, analyzer='char')

In [22]:
vect.fit(texts)

TfidfVectorizer(analyzer='char',
                vocabulary='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

In [23]:
len(vect.get_feature_names())

32

In [24]:
dataset = df_join_training_data.sample(frac=1).reset_index(drop=True)


In [25]:
text1_transform = vect.transform(dataset['text1'].values)

In [26]:
text2_transform = vect.transform(dataset['text2'].values)

In [27]:
X = text1_transform - text2_transform

In [28]:
y = dataset['same'].values

In [29]:
X.shape, y.shape

((52601, 32), (52601,))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)

In [31]:
clf = LogisticRegression()

In [32]:
clf.fit(X_train, y_train)

LogisticRegression()

In [33]:
preds = clf.predict(X_test)

In [34]:
f1_score(y_test, preds, average='macro')

0.36805121060573215

##### n grams

In [35]:
n_grams = 4

In [36]:
texts

array(['I shift a bit, warily letting my eyes dart from one owl to the other -- but my eyes are trained on the Barn Owl the most. Like Hoole...so like Hoole... He turns a bit, and our eyes meet directly. I can"t describe it...in this next moment, I don"t look away, how awkward it seems. I stare into his eyes. They"re like Hoole"s... They are Barn Owl eyes, but Hoole"s eyes. They"re his eyes...Hoole"s eyes... They hold that light of valor, justice, that one glow that I always made me feel my gizzard twitch in the bottom of my heart. Hoole... He never wanted me, did he? He loved me, but only who I was pretending to be. He never loved me, only Emerilla...lucky owl. I just wanted to belong -- because I was nothing -- and he gave me that. He was so fair, so kind...he gave me something even Kreeth could not. Mum and Da -- Ygryk and Pleek -- were supposed to be my parents. I was supposed to be their chick, Lutta. Yet they left me. They talked about me behind my back...I was a freak. My own pa

In [37]:
vect = TfidfVectorizer(max_features=3000, analyzer='char', ngram_range=(4,4))

In [ ]:
vect.fit(texts)

In [ ]:
len(vect.get_feature_names())

In [ ]:
dataset = df_join_training_data.sample(frac=1).reset_index(drop=True)

In [ ]:
dataset = dataset[:10000]

In [ ]:
text1_transform = vect.transform(dataset['text1'].values)
text2_transform = vect.transform(dataset['text2'].values)
X = text1_transform - text2_transform
y = dataset['same'].values
X.shape, y.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
f1_score(y_test, preds, average='macro')

In [ ]:
#!pip install scipy==1.5
#!pip install scikit-learn==0.24.1